In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/prices/prices.csv


In [2]:
!pip install backtrader backtrader_plotting


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 419.5/419.5 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.5/60.5 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 16.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - \ done
  Created wheel for bokeh: filename=bokeh-2.3.3-py3-none-any.whl size=11342786 sha256=0d2889dac17de4f230dc66a24414bd4f5c4c331d7e7dd14b8750ee74e226bacf
  Stored in directory: /root/.cache/pip/wheels/97/ad/e7/de4318fea6b68de95c71af5cfe0f12d724c4e7e6b649972eb8
Successfully built bokeh
  Attempting uninstall: bokeh
    Found existing installation: bokeh 2.4.3
    Uninstalling bokeh-2.4.3:
      Successfully uninstalled bokeh-2.4.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
geoviews 1.9.6 requires bokeh<2.5,>=2.4.0, but you have bokeh 2.3.3 which is incompatible.
panel 0.14.4

In [3]:
import backtrader as bt


In [4]:
df=pd.read_csv('/kaggle/input/prices/prices.csv')
df

,ticker,date,open,high,low,close
0,AXISBANK,2010-01-04,198.779999,199.990005,197.619995,140.712021
1,AXISBANK,2010-01-05,199.600006,204.000000,199.139999,143.647934
2,AXISBANK,2010-01-06,204.000000,205.740005,198.509995,141.236801
3,AXISBANK,2010-01-07,199.289993,201.199997,197.820007,142.236725
4,AXISBANK,2010-01-08,200.600006,203.979996,199.270004,144.094727
...,...,...,...,...,...,...
47568,SRTRANSFIN,2020-06-09,636.900024,650.950012,613.000000,619.599976
47569,SRTRANSFIN,2020-06-10,622.599976,641.700012,612.099976,635.000000
47570,SRTRANSFIN,2020-06-11,615.250000,676.000000,615.250000,637.900024
47571,SRTRANSFIN,2020-06-12,615.000000,693.700012,606.000000,684.400024


In [5]:
new_df=df.groupby('ticker')
# print(new_df.groups)

In [6]:
tickers=[]
for name, group in new_df:
    tickers.append(name)
    group['200sma']=group['close'].rolling(200).mean()
#     print()

In [7]:
ax_bnk=new_df.get_group('AXISBANK')
ax_bnk

,ticker,date,open,high,low,close
0,AXISBANK,2010-01-04,198.779999,199.990005,197.619995,140.712021
1,AXISBANK,2010-01-05,199.600006,204.000000,199.139999,143.647934
2,AXISBANK,2010-01-06,204.000000,205.740005,198.509995,141.236801
3,AXISBANK,2010-01-07,199.289993,201.199997,197.820007,142.236725
4,AXISBANK,2010-01-08,200.600006,203.979996,199.270004,144.094727
...,...,...,...,...,...,...
2567,AXISBANK,2020-06-09,434.950012,442.500000,416.500000,420.049988
2568,AXISBANK,2020-06-10,422.200012,432.700012,415.299988,427.450012
2569,AXISBANK,2020-06-11,426.500000,435.549988,408.700012,413.450012
2570,AXISBANK,2020-06-12,390.000000,412.500000,389.149994,408.000000


In [8]:
pip install cvxopt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.6/13.6 MB 91.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [9]:
import backtrader as bt
import pandas as pd
import numpy as np
from cvxopt import matrix, solvers
from datetime import datetime

class MyStrategy(bt.Strategy):
    params = (
        ('lookback_period', 200),  # Lookback period for SMA calculation
        ('risk_tolerance', 0.5)  # Risk tolerance level for portfolio optimization
    )

    def __init__(self):
        self.stocks = {}  # Dictionary to hold stock data
        self.month_end_flag = False  # Flag to indicate month end

    def next(self):
        if self.month_end_flag:
            self.rebalance()

    def notify_timer(self, timer, when, *args, **kwargs):
        if when == bt.Timer.SESSION_END:
            self.month_end_flag = True

    def rebalance(self):
        # Calculate the SMA for each stock
        sma_values = {stock: np.mean(stock_data.close.get(size=self.params.lookback_period)) for stock, stock_data in self.stocks.items()}

        # Filter stocks with price > 200 SMA
        selected_stocks = [stock for stock, sma in sma_values.items() if stock_data.close[0] > sma]

        # Prepare input for Mean Variance optimization
        selected_data = np.array([stock_data.close.get(size=self.params.lookback_period) for stock_data in self.stocks.values() if stock_data in selected_stocks])
        n = len(selected_stocks)
        expected_returns = np.mean(selected_data, axis=1)
        covariance_matrix = np.cov(selected_data)

        # Perform Mean Variance optimization
        P = matrix(covariance_matrix)
        q = matrix(np.zeros(n))
        G = matrix(-np.eye(n))
        h = matrix(np.zeros(n))
        A = matrix(np.ones(n), (1, n))
        b = matrix(1.0)
        sol = solvers.qp(P, q, G, h, A, b)
        weights = np.array(sol['x']).flatten()
        
        print(weights)  # Add this line to check the weights

        # Rebalance the portfolio based on weights
        for i, stock in enumerate(selected_stocks):
            self.order_target_percent(data=self.stocks[stock], target=weights[i])

        self.month_end_flag = False


# Read the data file containing stocks
df = pd.read_csv('/kaggle/input/prices/prices.csv')  # Replace 'data.csv' with your data file path

# Convert the date column to datetime format
df['date'] = pd.to_datetime(df['date'])

# Create a cerebro instance
cerebro = bt.Cerebro()

# Add the strategy
cerebro.addstrategy(MyStrategy)

# Iterate over each ticker
tickers = df['ticker'].unique()
for ticker in tickers:
    # Select data for the current ticker
    ticker_data = df[df['ticker'] == ticker].reset_index(drop=True)

    # Create a data feed
    data = bt.feeds.PandasData(dataname=ticker_data, datetime='date',
                               open='open', high='high', low='low', close='close',
                               volume=None)

    # Add the data feed to cerebro
    cerebro.adddata(data, name=ticker)

# Set the initial cash
cerebro.broker.setcash(1000000)

# Set the monthly timer
cerebro.add_timer(bt.Timer.SESSION_END, monthdays=[1])
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
# Run the backtest
cerebro.run()

print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
# Plot necessary graphs and calculate portfolio stats (e.g., using bt.analyzers)
# cerebro.plot()


Starting Portfolio Value: 1000000.00
Final Portfolio Value: 1000000.00


In [10]:
import backtrader as bt
import cvxopt as opt
import numpy as np
from cvxopt import blas, solvers
import pandas as pd

# Read data
data = bt.feeds.PandasData(dataname=pd.read_csv('/kaggle/input/prices/prices.csv', parse_dates=True, dtype={'open': float, 'high': float, 'low': float, 'close': float}))

# Group data by ticker
grouped_data = {}
for stock in data:
    ticker = stock.ticker
    if ticker not in grouped_data:
        grouped_data[ticker] = []
    grouped_data[ticker].append(stock)


# Strategy
class SMAStrategy(bt.Strategy):
    params = dict(sma_period=200, rebalance_month=1, lookback_period=252, ports_risk=0.1, slippage=0.01)

    def __init__(self):
        # Keep track of P, q, G, h, A, b for each ticker group
        self.P = []
        self.q = []
        self.G = []
        self.h = []
        self.A = []
        self.b = []

    def next(self):
        # Loop through each ticker group
        for ticker in grouped_data:
            # Get 200 SMA for the group
            sma = bt.indicators.SMA(grouped_data[ticker], period=self.params.sma_period)

            # Select stocks above 200 SMA
            selected_stocks = [s for s in grouped_data[ticker] if s.close[0] > sma[0]]

            # Mean-Variance Optimization to get weights
            stocks_ret = np.array([s.ret[0] for s in selected_stocks])
            stocks_cov = np.cov(stocks_ret[-self.params.lookback_period:])
            n = len(selected_stocks)

            # cvxopt matrices
            P = opt.matrix(stocks_cov)
            q = opt.matrix(np.zeros(n))
            G = opt.matrix(np.vstack((np.ones((1, n)), -np.eye(n))))
            h = opt.matrix(np.vstack((self.params.ports_risk, np.zeros((n, 1)))))
            A = opt.matrix(np.vstack((np.ones(n), stocks_ret.mean(axis=1)))).T
            b = opt.matrix([1.0, self.params.ports_risk])

            # Store optimization matrices
            self.P.append(P)
            self.q.append(q)
            self.G.append(G)
            self.h.append(h)
            self.A.append(A)
            self.b.append(b)

        # Rebalance portfolio at the start of the month
        if self.datas[0].datetime.date(0).month == self.params.rebalance_month:
            for i, selected_stocks in enumerate(grouped_data.values()):
                # Use cvxopt to solve QP Problem
                sol = solvers.qp(P=self.P[i], q=self.q[i], G=self.G[i], h=self.h[i],
                                 A=self.A[i], b=self.b[i])
                weights = np.array(sol['x']).reshape(n,)

                # Rebalance portfolio
                self.rebalance_portfolio(weights, selected_stocks)

    def rebalance_portfolio(self, weights, selected_stocks):
        # Calculate target shares
        target_shares = []
        for i, stock in enumerate(selected_stocks):
            target_shares.append((weights[i] * self.broker.getvalue()) / stock.close[0])

        # Rebalance portfolio with fixed slippage
        for i, stock in enumerate(selected_stocks):
            target = target_shares[i]
            current = self.getposition(stock).size if self.getposition(stock) else 0
            order_size = target - current
            self.order_target_percent(stock, target=target, exectype=bt.Order.Market,
                                      price=stock.close[0] * (1 + self.params.slippage))


# Initialize Cerebro
cerebro = bt.Cerebro()

# Add data feed
cerebro.adddata(data)

# Add strategy
cerebro.addstrategy(SMAStrategy)

# Set initial capital
cerebro.broker.setcash(100000)

# Set commission
cerebro.broker.setcommission(commission=0.001)

# Run the backtest
cerebro.run()

# Print final portfolio value
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())


AttributeError: 'int' object has no attribute 'to_pydatetime'

In [ ]:
import backtrader as bt
import pandas as pd
import numpy as np
from cvxopt import matrix, solvers
from datetime import datetime

class SMAStrategy(bt.Strategy):
    
    params = dict(sma_period=200, rebalance_month=1, lookback_period=252, ports_risk=0.1, slippage=0.01)

    def __init__(self):
        # Keep track of P, q, G, h, A, b for each ticker group
        self.P = []
        self.q = []
        self.G = []
        self.h = []
        self.A = []
        self.b = []

    def next(self):
        # Loop through each ticker group
        for ticker in grouped_data:
            # Get 200 SMA for the group
            sma = bt.indicators.SMA(grouped_data[ticker], period=self.params.sma_period)
            print(1)
            # Select stocks above 200 SMA
            selected_stocks = [s for s in grouped_data[ticker] if s.close[0] > sma[0]]

            # Mean-Variance Optimization to get weights
            stocks_ret = np.array([s.ret[0] for s in selected_stocks])
            stocks_cov = np.cov(stocks_ret[-self.params.lookback_period:])
            n = len(selected_stocks)

            # cvxopt matrices
            P = opt.matrix(stocks_cov)
            q = opt.matrix(np.zeros(n))
            G = opt.matrix(np.vstack((np.ones((1, n)), -np.eye(n))))
            h = opt.matrix(np.vstack((self.params.ports_risk, np.zeros((n, 1)))))
            A = opt.matrix(np.vstack((np.ones(n), stocks_ret.mean(axis=1)))).T
            b = opt.matrix([1.0, self.params.ports_risk])

            # Store optimization matrices
            self.P.append(P)
            self.q.append(q)
            self.G.append(G)
            self.h.append(h)
            self.A.append(A)
            self.b.append(b)

        # Rebalance portfolio at the start of the month
        if self.datas[0].datetime.date(0).month == self.params.rebalance_month:
            for i, selected_stocks in enumerate(grouped_data.values()):
                # Use cvxopt to solve QP Problem
                sol = solvers.qp(P=self.P[i], q=self.q[i], G=self.G[i], h=self.h[i],
                                 A=self.A[i], b=self.b[i])
                weights = np.array(sol['x']).reshape(n,)

                # Rebalance portfolio
                self.rebalance_portfolio(weights, selected_stocks)

    def rebalance_portfolio(self, weights, selected_stocks):
        # Calculate target shares
        target_shares = []
        for i, stock in enumerate(selected_stocks):
            target_shares.append((weights[i] * self.broker.getvalue()) / stock.close[0])

        # Rebalance portfolio with fixed slippage
        for i, stock in enumerate(selected_stocks):
            target = target_shares[i]
            current = self.getposition(stock).size if self.getposition(stock) else 0
            order_size = target - current
            self.order_target_percent(stock, target=target, exectype=bt.Order.Market,
                                      price=stock.close[0] * (1 + self.params.slippage))


# Read the data file containing stocks
df = pd.read_csv('/kaggle/input/prices/prices.csv')  # Replace 'data.csv' with your data file path

# Convert the date column to datetime format
df['date'] = pd.to_datetime(df['date'])

# Create a cerebro instance
cerebro = bt.Cerebro()

# Add the strategy
cerebro.addstrategy(SMAStrategy)

# Iterate over each ticker
tickers = df['ticker'].unique()
for ticker in tickers:
    # Select data for the current ticker
    ticker_data = df[df['ticker'] == ticker].reset_index(drop=True)

    # Create a data feed
    data = bt.feeds.PandasData(dataname=ticker_data, datetime='date',
                               open='open', high='high', low='low', close='close',
                               volume=None)

    # Add the data feed to cerebro
    cerebro.adddata(data, name=ticker)

# Set the initial cash
cerebro.broker.setcash(1000000)

# Set the monthly timer
cerebro.add_timer(bt.Timer.SESSION_END, monthdays=[1])
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
# Run the backtest
cerebro.run()

print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
# Plot necessary graphs and calculate portfolio stats (e.g., using bt.analyzers)
# cerebro.plot()


In [ ]:
import backtrader as bt
import pandas as pd
import numpy as np
from cvxopt import matrix, solvers
from datetime import datetime

class SMAStrategy(bt.Strategy):
    
    def log(self, txt, dt=None):
        ''' Logging function fot this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.datas[0].close

        # To keep track of pending orders and buy price/commission
        self.order = None
        self.buyprice = None
        self.buycomm = None

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                    (order.executed.price,
                     order.executed.value,
                     order.executed.comm))

                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:  # Sell
                self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                         (order.executed.price,
                          order.executed.value,
                          order.executed.comm))

            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        self.order = None

    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))

    def next(self):
        # Simply log the closing price of the series from the reference
        self.log('Close, %.2f' % self.dataclose[0])

        # Check if an order is pending ... if yes, we cannot send a 2nd one
        if self.order:
            return

        # Check if we are in the market
        if not self.position:

            # Not yet ... we MIGHT BUY if ...
            if self.dataclose[0] < self.dataclose[-1]:
                    # current close less than previous close

                    if self.dataclose[-1] < self.dataclose[-2]:
                        # previous close less than the previous close

                        # BUY, BUY, BUY!!! (with default parameters)
                        self.log('BUY CREATE, %.2f' % self.dataclose[0])

                        # Keep track of the created order to avoid a 2nd order
                        self.order = self.buy()

        else:

            # Already in the market ... we might sell
            if len(self) >= (self.bar_executed + 5):
                # SELL, SELL, SELL!!! (with all possible default parameters)
                self.log('SELL CREATE, %.2f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.sell()
        


# Read the data file containing stocks
df = pd.read_csv('/kaggle/input/prices/prices.csv')  # Replace 'data.csv' with your data file path

# Convert the date column to datetime format
df['date'] = pd.to_datetime(df['date'])

# Create a cerebro instance
cerebro = bt.Cerebro()

# Add the strategy
cerebro.addstrategy(SMAStrategy)

# Iterate over each ticker
tickers = df['ticker'].unique()
for ticker in tickers:
    # Select data for the current ticker
    ticker_data = df[df['ticker'] == ticker].reset_index(drop=True)

    # Create a data feed
    data = bt.feeds.PandasData(dataname=ticker_data, datetime='date',
                               open='open', high='high', low='low', close='close',
                               volume=None)

    # Add the data feed to cerebro
    cerebro.adddata(data, name=ticker)

# Set the initial cash
cerebro.broker.setcash(1000000)

# Set the monthly timer
cerebro.add_timer(bt.Timer.SESSION_END, monthdays=[1])
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
# Run the backtest
cerebro.run()

print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
# Plot necessary graphs and calculate portfolio stats (e.g., using bt.analyzers)
# cerebro.plot()


In [ ]:
import backtrader as bt
import pandas as pd
import numpy as np
from cvxopt import matrix, solvers
from datetime import datetime

class MyStrategy(bt.Strategy):
    def __init__(self):
        self.sma = bt.indicators.SimpleMovingAverage(self.data.close, period=200)

    def next(self):
#         Check if the current price is greater than the 200-day SMA
        if self.data.close[0] > self.sma[0]:
            self.buy()     


# Read the data file containing stocks
df = pd.read_csv('/kaggle/input/prices/prices.csv')  # Replace 'data.csv' with your data file path

# Convert the date column to datetime format
df['date'] = pd.to_datetime(df['date'])

# Create a cerebro instance
cerebro = bt.Cerebro()

# Add the strategy
cerebro.addstrategy(MyStrategy)

# Iterate over each ticker
tickers = df['ticker'].unique()
for ticker in tickers:
    # Select data for the current ticker
    ticker_data = df[df['ticker'] == ticker].reset_index(drop=True)

    # Create a data feed
    data = bt.feeds.PandasData(dataname=ticker_data, datetime='date',
                               open='open', high='high', low='low', close='close',
                               volume=None)

    # Add the data feed to cerebro
    cerebro.adddata(data, name=ticker)

# Set the initial cash
cerebro.broker.setcash(1000000)

# Set the monthly timer
cerebro.add_timer(bt.Timer.SESSION_END, monthdays=[1])
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
# Run the backtest
cerebro.run()

print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
# Plot necessary graphs and calculate portfolio stats (e.g., using bt.analyzers)
# cerebro.plot()


In [ ]:
import backtrader as bt
import pandas as pd
import numpy as np
from cvxopt import matrix, solvers
from datetime import datetime

class MyStrategy(bt.Strategy):
    def __init__(self):
        self.data_weights = self.calculate_weights()
        self.sma = [bt.indicators.SimpleMovingAverage(d.close, period=200) for d in self.datas]

    def calculate_weights(self):
        num_stocks = len(self.datas)
        weight = 1.0 / num_stocks
        return [weight] * num_stocks

    def next(self):
        # Check if the current price is greater than the 200-day SMA for each data feed
        for i, d in enumerate(self.datas):
            if d.close[0] > self.sma[i][0]:
                self.buy(data=d)  


# Read the data file containing stocks
df = pd.read_csv('/kaggle/input/prices/prices.csv')  # Replace 'data.csv' with your data file path

# Convert the date column to datetime format
df['date'] = pd.to_datetime(df['date'])

# Create a cerebro instance
cerebro = bt.Cerebro()

# Add the strategy
cerebro.addstrategy(MyStrategy)

# Iterate over each ticker
tickers = df['ticker'].unique()
for ticker in tickers:
    # Select data for the current ticker
    ticker_data = df[df['ticker'] == ticker].reset_index(drop=True)

    # Create a data feed
    data = bt.feeds.PandasData(dataname=ticker_data, datetime='date',
                               open='open', high='high', low='low', close='close',
                               volume=None)

    # Add the data feed to cerebro
    cerebro.adddata(data, name=ticker)

# Set the initial cash
cerebro.broker.setcash(1000000)

# Set the monthly timer
cerebro.add_timer(bt.Timer.SESSION_END, monthdays=[1])
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
# Run the backtest
cerebro.run()

print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
# Plot necessary graphs and calculate portfolio stats (e.g., using bt.analyzers)
# cerebro.plot()


In [ ]:
import backtrader as bt
import pandas as pd
import numpy as np
from cvxopt import matrix, solvers
from datetime import datetime

class MyStrategy(bt.Strategy):

    def __init__(self):
        self.lookback_period = 200
        self.data_weights = self.calculate_weights()
        self.sma = [bt.indicators.SimpleMovingAverage(d.close, period=self.lookback_period) for d in self.datas]

    def calculate_weights(self):
        num_stocks = len(self.datas)
        num_assets = num_stocks
        lookback_period = min(self.lookback_period, min(len(d) for d in self.datas))
        prices = np.zeros((num_assets, lookback_period))

        # Check if all stocks have sufficient historical data
        if lookback_period == 0:
            return [1 / num_stocks] * num_stocks

        # Collect historical prices for each stock
        for i, d in enumerate(self.datas):
            if len(d) >= lookback_period:  # Skip stocks with insufficient historical data
                prices[i] = d.close.get(size=lookback_period)
        
        returns = (prices[:, -1] / prices[:, 0]) - 1  # Calculate returns based on the lookback period

        # Mean Variance Optimization using CVXOPT
        returns = matrix(returns)
        cov_matrix = matrix(np.cov(prices))
        P = cov_matrix
        q = matrix(np.zeros(num_assets))
        G = matrix(-np.eye(num_assets))
        h = matrix(np.zeros(num_assets))
        A = matrix(np.ones(num_assets), (1, num_assets))
        b = matrix(1.0)
        
        # Solve the optimization problem
        sol = solvers.qp(P, q, G, h, A, b)
        weights = np.array(sol['x']).flatten()
        print(1)
        return weights.tolist()

    def next(self):
        # Check if the current price is greater than the 200-day SMA for each data feed
        for i, d in enumerate(self.datas):
            if d.close[0] > self.sma[i][0]:
                self.buy(data=d)  


# Read the data file containing stocks
df = pd.read_csv('/kaggle/input/prices/prices.csv')  # Replace 'data.csv' with your data file path

# Convert the date column to datetime format
df['date'] = pd.to_datetime(df['date'])

# Create a cerebro instance
cerebro = bt.Cerebro()

# Add the strategy
cerebro.addstrategy(MyStrategy)

# Iterate over each ticker
tickers = df['ticker'].unique()
for ticker in tickers:
    # Select data for the current ticker
    ticker_data = df[df['ticker'] == ticker].reset_index(drop=True)

    # Create a data feed
    data = bt.feeds.PandasData(dataname=ticker_data, datetime='date',
                               open='open', high='high', low='low', close='close',
                               volume=None)

    # Add the data feed to cerebro
    cerebro.adddata(data, name=ticker)

# Set the initial cash
cerebro.broker.setcash(1000000)

# Set the monthly timer
cerebro.add_timer(bt.Timer.SESSION_END, monthdays=[1])
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
# Run the backtest
cerebro.run()

print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
# Plot necessary graphs and calculate portfolio stats (e.g., using bt.analyzers)
# cerebro.plot()


In [ ]:
import backtrader as bt
import pandas as pd
import numpy as np
from cvxopt import matrix, solvers
from datetime import datetime

class MyStrategy(bt.Strategy):
    def __init__(self):
        self.lookback_period = 200
        self.data_weights = self.calculate_weights()
        self.sma = [bt.indicators.SimpleMovingAverage(d.close, period=self.lookback_period) for d in self.datas]
        self.add_timer(bt.timer.SESSION_START, monthdays=[1])  # Schedule the reweighting function

    def calculate_weights(self):
        num_stocks = len(self.datas)
        num_assets = num_stocks
        lookback_period = min(self.lookback_period, min(len(d) for d in self.datas))
        prices = np.zeros((num_assets, lookback_period))

        # Check if all stocks have sufficient historical data
        if lookback_period == 0:
            return [1 / num_stocks] * num_stocks

        # Collect historical prices for each stock
        for i, d in enumerate(self.datas):
            if len(d) >= lookback_period:  # Skip stocks with insufficient historical data
                prices[i] = d.close.get(size=lookback_period)

        returns = (prices[:, -1] / prices[:, 0]) - 1  # Calculate returns based on the lookback period

        # Mean Variance Optimization using CVXOPT
        returns = matrix(returns)
        cov_matrix = matrix(np.cov(prices))
        P = cov_matrix
        q = matrix(np.zeros(num_assets))
        G = matrix(-np.eye(num_assets))
        h = matrix(np.zeros(num_assets))
        A = matrix(np.ones(num_assets), (1, num_assets))
        b = matrix(1.0)

        # Solve the optimization problem
        sol = solvers.qp(P, q, G, h, A, b)
        weights = np.array(sol['x']).flatten()

        return weights.tolist()

    def reweight(self):
        self.data_weights = self.calculate_weights()

    def reweight_monthly(self):
        if self.datetime.date().day == 1:  # Check if it's the first day of the month
            self.reweight()

    def next(self):
        self.reweight_monthly()  # Call the reweighting function at the start of each month

        for i, d in enumerate(self.datas):
            if self.data_weights[i] > 0 and d.close[0] > self.sma[i][0]:
                self.order_target_percent(d, target=self.data_weights[i]) 


# Read the data file containing stocks
df = pd.read_csv('/kaggle/input/prices/prices.csv')  # Replace 'data.csv' with your data file path

# Convert the date column to datetime format
df['date'] = pd.to_datetime(df['date'])

# Create a cerebro instance
cerebro = bt.Cerebro()

# Add the strategy
cerebro.addstrategy(MyStrategy)

# Iterate over each ticker
tickers = df['ticker'].unique()
for ticker in tickers:
    # Select data for the current ticker
    ticker_data = df[df['ticker'] == ticker].reset_index(drop=True)

    # Create a data feed
    data = bt.feeds.PandasData(dataname=ticker_data, datetime='date',
                               open='open', high='high', low='low', close='close',
                               volume=None)

    # Add the data feed to cerebro
    cerebro.adddata(data, name=ticker)

# Set the initial cash
cerebro.broker.setcash(1000000)

# Set the monthly timer
cerebro.add_timer(bt.Timer.SESSION_END, monthdays=[1])
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
# Run the backtest
cerebro.run()

print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
# Plot necessary graphs and calculate portfolio stats (e.g., using bt.analyzers)
# cerebro.plot()


In [ ]:
import backtrader as bt
import pandas as pd
import numpy as np
from cvxopt import matrix, solvers
from datetime import datetime

class MyStrategy(bt.Strategy):
    def __init__(self):
        self.lookback_period = 252
        self.data_weights = self.calculate_weights()
        self.sma = [bt.indicators.SimpleMovingAverage(d.close, period=200) for d in self.datas]
        self.add_timer(bt.timer.SESSION_START, monthdays=[1])  # Schedule the reweighting function

    def calculate_weights(self):
        num_stocks = len(self.datas)
        num_assets = num_stocks
        lookback_period = min(self.lookback_period, min(len(d) for d in self.datas))
        prices = np.zeros((num_assets, lookback_period))

        # Check if all stocks have sufficient historical data
        if lookback_period == 0:
            return [1 / num_stocks] * num_stocks

        # Collect historical prices for each stock
        for i, d in enumerate(self.datas):
            if len(d) >= lookback_period:  # Skip stocks with insufficient historical data
                prices[i] = d.close.get(size=lookback_period)

        returns = (prices[:, -1] / prices[:, 0]) - 1  # Calculate returns based on the lookback period

        # Mean Variance Optimization using CVXOPT
        returns = matrix(returns)
        cov_matrix = matrix(np.cov(prices))
        P = cov_matrix
        q = matrix(np.zeros(num_assets))
        G = matrix(-np.eye(num_assets))
        h = matrix(np.zeros(num_assets))
        A = matrix(np.ones(num_assets), (1, num_assets))
        b = matrix(1.0)

        # Solve the optimization problem
        sol = solvers.qp(P, q, G, h, A, b)
        weights = np.array(sol['x']).flatten()

        return weights.tolist()

    def reweight(self):
        self.data_weights = self.calculate_weights()

    def reweight_monthly(self):
        if self.datetime.date().day == 1:  # Check if it's the first day of the month
            self.reweight()

    def next(self):
        self.reweight_monthly()  # Call the reweighting function at the start of each month

        for i, d in enumerate(self.datas):
            if self.data_weights[i] > 0 and d.close[0] > self.sma[i][0]:
                self.order_target_percent(d, target=self.data_weights[i]) 


# Read the data file containing stocks
df = pd.read_csv('/kaggle/input/prices/prices.csv')  # Replace 'data.csv' with your data file path

# Convert the date column to datetime format
df['date'] = pd.to_datetime(df['date'])

# Create a cerebro instance
cerebro = bt.Cerebro()

# Add the strategy
cerebro.addstrategy(MyStrategy)

# Iterate over each ticker
tickers = df['ticker'].unique()
for ticker in tickers:
    # Select data for the current ticker
    ticker_data = df[df['ticker'] == ticker].reset_index(drop=True)

    # Create a data feed
    data = bt.feeds.PandasData(dataname=ticker_data, datetime='date',
                               open='open', high='high', low='low', close='close',
                               volume=None)

    # Add the data feed to cerebro
    cerebro.adddata(data, name=ticker)

# Set the initial cash
cerebro.broker.setcash(1000000)

# Set the monthly timer
cerebro.add_timer(bt.Timer.SESSION_END, monthdays=[1])
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
# Run the backtest
cerebro.run()

print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
# Plot necessary graphs and calculate portfolio stats (e.g., using bt.analyzers)
# cerebro.plot()


In [ ]:
import backtrader as bt
import pandas as pd
import numpy as np
from cvxopt import matrix, solvers
from datetime import datetime

class MyStrategy(bt.Strategy):
    def __init__(self):
        self.lookback_period = 300
        self.data_weights = self.calculate_weights()
        self.sma = [bt.indicators.SimpleMovingAverage(d.close, period=200) for d in self.datas]
        self.add_timer(bt.timer.SESSION_START, monthdays=[1])  # Schedule the reweighting function

    def calculate_weights(self):
        num_stocks = len(self.datas)
        num_assets = num_stocks
        lookback_period = min(self.lookback_period, min(len(d) for d in self.datas))
        prices = np.zeros((num_assets, lookback_period))

        # Check if all stocks have sufficient historical data
        if lookback_period == 0:
            return [1 / num_stocks] * num_stocks

        # Collect historical prices for each stock
        for i, d in enumerate(self.datas):
            if len(d) >= lookback_period:  # Skip stocks with insufficient historical data
                prices[i] = d.close.get(size=lookback_period)

        returns = (prices[:, -1] / prices[:, 0]) - 1  # Calculate returns based on the lookback period

        # Mean Variance Optimization using CVXOPT
        returns = matrix(returns)
        cov_matrix = matrix(np.cov(prices))
        P = cov_matrix
        q = matrix(np.zeros(num_assets))
        G = matrix(-np.eye(num_assets))
        h = matrix(np.zeros(num_assets))
        A = matrix(np.ones(num_assets), (1, num_assets))
        b = matrix(1.0)

        # Solve the optimization problem
        sol = solvers.qp(P, q, G, h, A, b)
        weights = np.array(sol['x']).flatten()

        return weights.tolist()

    def reweight(self):
        self.data_weights = self.calculate_weights()

    def reweight_monthly(self):
        if self.datetime.date().day == 1:  # Check if it's the first day of the month
            self.reweight()

    def next(self):
        self.reweight_monthly()  # Call the reweighting function at the start of each month

        for i, d in enumerate(self.datas):
            if self.data_weights[i] > 0 and d.close[0] > self.sma[i][0]:
                self.order_target_percent(d, target=self.data_weights[i]) 


# Read the data file containing stocks
df = pd.read_csv('/kaggle/input/prices/prices.csv')  # Replace 'data.csv' with your data file path

# Convert the date column to datetime format
df['date'] = pd.to_datetime(df['date'])

# Create a cerebro instance
cerebro = bt.Cerebro()

# Add the strategy
cerebro.addstrategy(MyStrategy)

# Iterate over each ticker
tickers = df['ticker'].unique()
for ticker in tickers:
    # Select data for the current ticker
    ticker_data = df[df['ticker'] == ticker].reset_index(drop=True)

    # Create a data feed
    data = bt.feeds.PandasData(dataname=ticker_data, datetime='date',
                               open='open', high='high', low='low', close='close',
                               volume=None)

    # Add the data feed to cerebro
    cerebro.adddata(data, name=ticker)

# Set the initial cash
cerebro.broker.setcash(1000000)

# Set the monthly timer
cerebro.add_timer(bt.Timer.SESSION_END, monthdays=[1])
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
# Run the backtest
cerebro.run()

print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
# Plot necessary graphs and calculate portfolio stats (e.g., using bt.analyzers)
# cerebro.plot()
